In [1]:
import numpy as np
from scipy.linalg import expm, eig, solve, lstsq
import matplotlib.pyplot as plt

In [2]:
x = np.array([[0,0.5],[0.5,0]])
y = np.array([[0,-0.5j],[0.5j,0]])
z = np.array([[0.5,0],[0,-0.5]])
sx = np.kron(x, np.identity(2))
sy = np.kron(y, np.identity(2))
sz = np.kron(z, np.identity(2))
ix = np.kron(np.identity(2), x)
iy = np.kron(np.identity(2), y)
iz = np.kron(np.identity(2), z)

In [3]:
ham_sz = 400e6 * np.kron(z, np.identity(2))
ham_iz = 400e6 * np.kron(np.identity(2), z)
ham_hf = 2e6 * np.kron(z, x)
ham_emr = 1e6 * np.kron(x, np.identity(2))

In [4]:
ham_offset = 263e9 * np.kron(z, np.identity(2))

In [5]:
ham_int = ham_sz + ham_iz + ham_hf
ham_eff = ham_int + ham_emr

In [6]:
ham0 = ham_int + ham_offset

In [7]:
ham_lab = ham_eff + ham_offset

In [8]:
h = 6.62607004e-34
k = 1.38064852e-23
T = 100 # temperature

In [9]:
rho0 = expm(-(h/(k*T)) * ham0)/np.trace(expm(-(h/(k*T)) * ham0))

In [10]:
rho_eq = expm(-(h/(k*T)) * ham_lab)/np.trace(expm(-(h/(k*T)) * ham_lab))

In [11]:
rho_eq_super = rho_eq.flatten()[np.newaxis].T

In [12]:
dt = 1e2/263e9

In [13]:
D = expm(-2.0j * np.pi * ham_offset * dt)
print(D)

[[1.-1.96438672e-15j 0.+0.00000000e+00j 0.+0.00000000e+00j
  0.+0.00000000e+00j]
 [0.+0.00000000e+00j 1.-1.96438672e-15j 0.+0.00000000e+00j
  0.+0.00000000e+00j]
 [0.+0.00000000e+00j 0.+0.00000000e+00j 1.+1.96438672e-15j
  0.+0.00000000e+00j]
 [0.+0.00000000e+00j 0.+0.00000000e+00j 0.+0.00000000e+00j
  1.+1.96438672e-15j]]


In [14]:
def commutationSuperOp(op):
    res = np.kron(op, np.identity(op.shape[0])) - np.kron(np.identity(op.shape[0]), op.T)
    return res

In [15]:
h_super = commutationSuperOp(ham_eff)

In [16]:
eigenval, eigenvec = eig(ham_lab)

In [17]:
t1e = 1.0e-3
t2e = 1.0e-6
t1n = 10
t2n = 4.0e-3

In [18]:
def secularRelaxationSuperOp(op):
    Aq = commutationSuperOp(op)
    Anq = commutationSuperOp(op.conjugate().T)
    return np.dot(Anq, Aq)

In [19]:
def t1SuperOp(t1, eigenvec, x, y):
    x_rotated = np.dot(eigenvec, np.dot(x, eigenvec.conjugate().T))
    y_rotated = np.dot(eigenvec, np.dot(y, eigenvec.conjugate().T))
    
    part_x = secularRelaxationSuperOp(x_rotated)
    part_y = secularRelaxationSuperOp(y_rotated)
    
    return 0.5 * 1.0/t1 * (part_x + part_y)

In [20]:
def t2SuperOp(t2, eigenvec, z):
    z_rotated = np.dot(eigenvec, np.dot(z, eigenvec.conjugate().T))
    
    part_z = secularRelaxationSuperOp(z_rotated)
    
    return 1.0/t2 * part_z

In [21]:
gamma_t1e = t1SuperOp(t1e, eigenvec, sx, sy)
gamma_t1n = t1SuperOp(t1n, eigenvec, ix, iy)
gamma_t2e = t2SuperOp(t2e, eigenvec, sz)
gamma_t2n = t2SuperOp(t2n, eigenvec, iz)

In [22]:
gamma_lab = gamma_t1e + gamma_t2e + gamma_t1n + gamma_t2n

In [23]:
L = 1.0j * h_super + gamma_lab

In [24]:
gamma_rho_eq_super = np.dot(gamma_lab, rho_eq_super)

In [25]:
results = lstsq(L, gamma_rho_eq_super)

In [43]:
L

array([[ 5.00050788e+02+0.e+00j, -3.12435547e-01-5.e+05j,
        -1.89730448e+00-5.e+05j,  6.01144126e-06+0.e+00j,
        -3.12435547e-01+5.e+05j, -5.07810889e-02+0.e+00j,
        -3.60847050e-09+0.e+00j, -1.92786274e-07+0.e+00j,
        -1.89730448e+00+5.e+05j, -3.60847050e-09+0.e+00j,
        -5.00000007e+02+0.e+00j,  7.69642354e-11+0.e+00j,
         6.01144126e-06+0.e+00j, -1.92786274e-07+0.e+00j,
         7.69642354e-11+0.e+00j, -7.32000993e-13+0.e+00j],
       [-3.12435547e-01-5.e+05j,  7.50049226e+02+4.e+08j,
         6.02970919e-06+0.e+00j, -1.89920273e+00-5.e+05j,
        -7.81088868e-04+0.e+00j,  3.12435547e-01+5.e+05j,
        -2.96991964e-09+0.e+00j,  3.60921458e-09+0.e+00j,
         3.59827391e-09+0.e+00j, -1.89920273e+00+5.e+05j,
         2.71651636e-11+0.e+00j,  4.99999993e+02+0.e+00j,
        -2.96091307e-09+0.e+00j,  8.38017100e-06+0.e+00j,
        -1.13620986e-14+0.e+00j, -2.23263735e-11+0.e+00j],
       [-1.89730448e+00-5.e+05j,  6.02970919e-06+0.e+00j,
         1.0

In [42]:
for i in range(L.shape[0]):
    print('{\n  ', end='')
    for j in range(L.shape[1]-1):
        print('cxdbl({}, {}), '.format(L[i,j].real, L[i,j].imag), end='')
    print('cxdbl({}, {}), '.format(L[i,L.shape[1]-1].real, L[i,L.shape[1]-1].imag), end='')
    print('\n},')

{
  cxdbl(500.0507882919981, 0.0), cxdbl(-0.3124355472758323, -500000.0), cxdbl(-1.897304479874008, -500000.0), cxdbl(6.01144125944056e-06, 0.0), cxdbl(-0.3124355472758323, 500000.0), cxdbl(-0.050781088867451814, 0.0), cxdbl(-3.608470504652113e-09, 0.0), cxdbl(-1.927862737394476e-07, 0.0), cxdbl(-1.897304479874008, 500000.0), cxdbl(-3.608470504652113e-09, 0.0), cxdbl(-500.00000720312994, 0.0), cxdbl(7.696423535276798e-11, 0.0), cxdbl(6.01144125944056e-06, 0.0), cxdbl(-1.927862737394476e-07, 0.0), cxdbl(7.696423535276798e-11, 0.0), cxdbl(-7.320009932344586e-13, 0.0), 
},
{
  cxdbl(-0.3124355472758323, -500000.0), cxdbl(750.0492261214687, 400000000.0), cxdbl(6.029709186910314e-06, 0.0), cxdbl(-1.8992027334806911, -500000.0), cxdbl(-0.0007810888681724938, 0.0), cxdbl(0.3124355472709935, 500000.0), cxdbl(-2.969919640254635e-09, 0.0), cxdbl(3.609214576170885e-09, 0.0), cxdbl(3.598273912084254e-09, 0.0), cxdbl(-1.8992027334896961, 500000.0), cxdbl(2.7165163571595614e-11, 0.0), cxdbl(499.9999

In [127]:
gamma_rho_eq_super - np.dot(L, results[0])

array([[-2.10850004e-10+1.16951782e-10j],
       [-1.92493346e-10+2.36468622e-11j],
       [-5.93825649e-11-1.20537125e-10j],
       [-1.21291574e-11-4.83169060e-12j],
       [-1.92711126e-10+5.59339242e-11j],
       [-1.76321180e-11+2.02362571e-11j],
       [-7.53973136e-12+7.35056460e-12j],
       [-3.63125384e-10-1.14444364e-10j],
       [-5.91749285e-11-8.74255460e-11j],
       [-1.81730882e-12-1.34114941e-11j],
       [-2.26135555e-10+1.16664012e-10j],
       [ 2.65388507e-10-3.09796633e-11j],
       [ 1.56839069e-11+5.19406740e-12j],
       [-4.33855501e-10-3.69632207e-11j],
       [ 1.16596635e-10+1.65528036e-10j],
       [ 4.54605242e-10-2.53852050e-10j]])

In [44]:
for i in range(gamma_rho_eq_super.shape[0]):
    print('{\n  ', end='')
    for j in range(gamma_rho_eq_super.shape[1]-1):
        print('cxdbl({}, {}), '.format(gamma_rho_eq_super[i,j].real, gamma_rho_eq_super[i,j].imag), end='')
    print('cxdbl({}, {}), '.format(gamma_rho_eq_super[i,gamma_rho_eq_super.shape[1]-1].real, 
                                   gamma_rho_eq_super[i,gamma_rho_eq_super.shape[1]-1].imag), end='')
    print('\n},')

{
  cxdbl(-15.778991805212845, 0.0), 
},
{
  cxdbl(3.7811030673545742e-06, 0.0), 
},
{
  cxdbl(-5.990504709861602e-05, 0.0), 
},
{
  cxdbl(2.27084151437427e-10, 0.0), 
},
{
  cxdbl(3.7811030673406965e-06, 0.0), 
},
{
  cxdbl(-15.78201668892785, 0.0), 
},
{
  cxdbl(2.2831399407883732e-10, 0.0), 
},
{
  cxdbl(-5.9916549312110007e-05, 0.0), 
},
{
  cxdbl(-5.9905047098670805e-05, 0.0), 
},
{
  cxdbl(2.2831399407883732e-10, 0.0), 
},
{
  cxdbl(15.77898700596819, 0.0), 
},
{
  cxdbl(3.793101180246672e-06, 0.0), 
},
{
  cxdbl(2.27084151437427e-10, 0.0), 
},
{
  cxdbl(-5.9916549312097626e-05, 0.0), 
},
{
  cxdbl(3.793101180246672e-06, 0.0), 
},
{
  cxdbl(15.782021488172493, 0.0), 
},


In [128]:
results[0]

array([[-3.08750557e-02+2.95180996e-11j],
       [-3.79844430e-05+1.51704299e-09j],
       [-3.91989394e-05-9.94146044e-08j],
       [ 1.51796015e-09+1.28224464e-10j],
       [-3.79844430e-05-1.51722469e-09j],
       [-4.87401817e-04+1.02199915e-10j],
       [-9.80020549e-08+1.21358606e-06j],
       [-3.92064659e-05-9.94334018e-08j],
       [-3.91989394e-05+9.94146038e-08j],
       [-9.80022359e-08-1.21358606e-06j],
       [ 4.84391199e-04+2.97430196e-11j],
       [ 3.79919695e-05-1.51704277e-09j],
       [ 1.51796015e-09-1.28224690e-10j],
       [-3.92064659e-05+9.94334005e-08j],
       [ 3.79919695e-05+1.51722515e-09j],
       [ 3.08780664e-02+1.02696510e-10j]])

In [46]:
for i in range(results[0].shape[0]):
    print('{\n  ', end='')
    for j in range(results[0].shape[1]-1):
        print('cxdbl({}, {}), '.format(results[0][i,j].real, results[0][i,j].imag), end='')
    print('cxdbl({}, {}), '.format(results[0][i,results[0].shape[1]-1].real, 
                                   results[0][i,results[0].shape[1]-1].imag), end='')
    print('\n},')

{
  cxdbl(-0.030875055742808813, 2.9518099639110554e-11), 
},
{
  cxdbl(-3.798444301407042e-05, 1.5170429889616723e-09), 
},
{
  cxdbl(-3.919893940999409e-05, -9.941460438025703e-08), 
},
{
  cxdbl(1.5179601483929651e-09, 1.2822446385072126e-10), 
},
{
  cxdbl(-3.7984443014296486e-05, -1.517224693785075e-09), 
},
{
  cxdbl(-0.00048740181672443845, 1.0219991508927755e-10), 
},
{
  cxdbl(-9.800205485118254e-08, 1.2135860550272787e-06), 
},
{
  cxdbl(-3.9206465896110466e-05, -9.943340175500611e-08), 
},
{
  cxdbl(-3.919893940976975e-05, 9.941460381767523e-08), 
},
{
  cxdbl(-9.80022358636542e-08, -1.2135860550246832e-06), 
},
{
  cxdbl(0.0004843911986832152, 2.974301956984995e-11), 
},
{
  cxdbl(3.7991969547203735e-05, -1.5170427664939994e-09), 
},
{
  cxdbl(1.5179601475486182e-09, -1.2822469031258052e-10), 
},
{
  cxdbl(-3.920646589588768e-05, 9.943340051341728e-08), 
},
{
  cxdbl(3.799196954743034e-05, 1.517225150878216e-09), 
},
{
  cxdbl(0.030878066351274228, 1.0269651033665897e-10), 

In [129]:
acq_e, acq_h = np.kron(z, np.identity(2)), np.kron(np.identity(2), z)

In [130]:
acq_e = acq_e.flatten()[np.newaxis]
acq_h = acq_h.flatten()[np.newaxis]

In [131]:
result = results[0]

In [132]:
val_e = np.dot(acq_e.conjugate(), result)
val_h = np.dot(acq_h.conjugate(), result)
print(val_e, val_h)

[[-0.03136246-3.60757589e-13j]] [[-0.03039066-7.28176531e-11j]]
